In [1]:
# cv to optimize hyperparameter K (window len) (in principle we expect an avg len of SP of 22)
# hyperparameters are K, gamma(sd of gaussian kernel) and c
# POSSIBLE FEATURES
# hydrophobicity profile along the SP (sliding window of 5), after we extract only the avg and max hp from the profile; encode also pos in which there is the max value, 
# soe cases of SP are enriched of + resdues in the N-terminal

# normalize hp avg and max????
# add also positivily charged max and pos features?

In [2]:
import numpy as np
import pandas as pd
import re
from scipy.stats import sem
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.metrics import matthews_corrcoef,precision_score, recall_score,accuracy_score
from Bio.SeqUtils.ProtParam import ProteinAnalysis as pa

def get_modes(df):
    modes = df['K'].mode()
    best_index = 0
    if len(modes) >1:
        diff = (modes[0] - 23)**2
        print(diff)
        for i in range(1,len(modes)):
            if (modes[i]-23)**2 <= diff:
                best_index = i
    best_K = modes[best_index]
    # scale is preferred over other gamma values
    modes = df['gamma'].mode()
    best_gamma = modes[0]
    for mode in modes:
        if mode == 'scale':
            best_gamma = 'scale'
    # lower value of C generalize better
    modes = df['C'].mode()
    best_C = modes[0]
    for i in range(1,len(modes)):
        if modes[i]<best_C:
            best_C = modes[i]
    return(best_K,best_gamma,best_C)

def get_results(df,name_of_model):
    hp_modes = get_modes(df)
    avg = df[['MCC val','MCC test','ACC','precision','recall']].mean()
    se = df[['MCC val','MCC test','ACC','precision','recall']].sem()
    # Create a new DataFrame with the desired format
    result_df = pd.DataFrame({
        'metric': ['MCC val','MCC test','ACC','precision','recall'],
        name_of_model: [f'{avg:.2f} ± ({std:.2f})' for avg, std in zip(avg, se)]})

    #Set the 'metric' column as the index
    result_df.set_index('metric', inplace=True)
    result_df = result_df.transpose()[:][:]
    result_df.columns = ['MCC val','MCC test','ACC','precision','recall']
    result_df.insert(0, 'K', hp_modes[0])
    result_df.insert(1, 'gamma', hp_modes[1])
    result_df.insert(2, 'C', hp_modes[2])
    result_df = result_df.reset_index(names='model')
    return result_df


def encode(df,K,c_comp=False,local_hp=False,local_hp2=False,K2=False,win=False,features_filter=False,ch=False):
    #add X = 0 to avoid warning regarding unknown aa
    hp_scale = {'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5, 'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5, 'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6, 'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2, 'X':0.0, 'Z': 0.0, 'B': 0,'U':0}
    
    ch_scale = {"A": 0, "R": 1, "N": 0, "D": 0, "C": 0, "Q": 0, "E": 0, "G": 0, "H": 1, "I": 0, "L": 0, "K": 1, "M": 0, "F": 0, "P": 0, "S": 0, "T": 0, "W": 0, "Y": 0, "V": 0,'X':0,'Z':0,'B':0}
    
    columns = ['A1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1', 'K1', 'L1', 'M1', 'N1', 'P1', 'Q1', 'R1', 'S1', 'T1', 'V1', 'W1', 'Y1']

    # n_term comosition-------------------------------------------------------------------------------------------------------------------
    n_comp = np.empty((0, 20)) 
    for index,row in df.iterrows():
        seq = pa(row['Sequence'][:K])
        profile = np.array(list(seq.get_amino_acids_percent().values()))
        n_comp = np.vstack((n_comp,profile))
    enc_array = n_comp
    
    # c_term composition-----------------------------------------------------------------------------------------------------------------
    if c_comp:
        c_comp = np.empty((0, 20)) 
        for index,row in df.iterrows():
            seq = pa(row['Sequence'][K:])
            profile = np.array(list(seq.get_amino_acids_percent().values()))
            c_comp = np.vstack((c_comp,profile))
        enc_array = np.hstack((enc_array,c_comp))
        columns.extend(['A2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2', 'K2', 'L2', 'M2', 'N2', 'P2', 'Q2', 'R2', 'S2', 'T2', 'V2', 'W2', 'Y2'])
    # local hydrophobicity window(7) from 0 to K -> avg, max and argmax(pos)-----------------------------------------------------------
    if local_hp:
        local_hp = np.empty((0, 3)) 
        for index,row in df.iterrows():
            profile = pa.protein_scale(pa(row['Sequence'][:K]),param_dict=hp_scale, window=7, edge=1.0)  #7
            features = np.array([np.argmax(profile)/K, np.max(profile),np.average(profile)])
            local_hp = np.vstack((local_hp,features))
        local_hp[:,1] = (local_hp[:,1]+4.5)/9      #normalize
        local_hp[:,2] = (local_hp[:,2]+4.5)/9
        enc_array = np.hstack((enc_array,local_hp))
        columns.extend(['HP max pos','HP max','HP avg'])

    # local hydrophobicity window(12) from 0 to K2 -> avg, max and argmax(pos)

    if local_hp2:
        local_hp2 = np.empty((0, 3)) 
        for index,row in df.iterrows():
            profile = pa.protein_scale(pa(row['Sequence'][:K2]),param_dict=hp_scale, window=win, edge=1.0)  #12
            features = np.array([np.argmax(profile)/K2, np.max(profile),np.average(profile)])
            local_hp2 = np.vstack((local_hp2,features))
        local_hp2[:,1] = (local_hp2[:,1]+4.5)/9    #normalize
        local_hp2[:,2] = (local_hp2[:,2]+4.5)/9
        enc_array = np.hstack((enc_array,local_hp2))    
        columns.extend(['HP2 max pos','HP2 max','HP2 avg'])   
    
    if ch:
        ch = np.empty((0, 3)) 
        for index,row in df.iterrows():
            profile = pa.protein_scale(pa(row['Sequence'][:K]),param_dict=ch_scale, window=5, edge=1.0)  
            features = np.array([np.argmax(profile)/K, np.max(profile),np.average(profile)])
            ch = np.vstack((ch,features))
        enc_array = np.hstack((enc_array,ch))  
        columns.extend(['CH max pos','CH max','CH avg'])   

    enc_array = pd.DataFrame(enc_array, columns=columns)
    if features_filter != False: enc_array = enc_array[features_filter]
    return enc_array

### Datasets preparation

In [3]:
#! conda install biopython -y
#random forest to perform feature selection
#to search scales by code
#https://www.genome.jp/aaindex/
training= pd.read_table('../stats/training_metadata_parsed.tsv')[['Signal peptide','Sequence']]
training = training.sample(frac=1, random_state=3)
training['class'] = 0
training.loc[training['Signal peptide'].notna(), 'class'] = 1
#split + and - in 2 df
positives = training.loc[training['class'] == 1,][['Sequence','class']]
negatives = training.loc[training['class'] == 0,][['Sequence','class']]
#positives=np.array(positives)
positives = np.array_split(positives,5)
negatives = np.array_split(negatives,5)
#pa.protein_scale(scale_dict, sliding_window_size)

### Grid search CV (optimize K, gamma and C)

In [4]:
#feature encoding funtion parameters
model_combination = 'N-COMP+HP2+C-COMP+FF'
c_comp = True #
local_hp = False
local_hp2 = True
ch = False # useless
#best features obtained with permutation importance
#features_filter = False
features_filter = ['HP2 max','HP2 max pos','L1','HP2 avg','R1','W1','K1','K2','I1','D1','S1','E1','I2','G1','C2','M2','R2','L2','G2','Q1','N1','Y2','V1','N2','D2','P1','Y1','H1']
# minimal grid search
C_list = [1,2,4,8,16]
gamma_list = [1,2,4,8,16,'scale']
K_list = [18,19,20,21,22,23,24,25,26,27,28]
#
C_list = [8]
gamma_list = [8]
K_list = [27]
#
K2=40
win=12
###
col_names = ['run','training','validation','testing','K','gamma','C','MCC val','MCC test','ACC','precision','recall']
cv_results = []

for run in range(5):
    hyperparameter_table =[]
    for K in K_list:
        training = pd.concat((positives[(0+run)%5],positives[(1+run)%5],positives[(2+run)%5],negatives[(0+run)%5],negatives[(1+run)%5],negatives[(2+run)%5]),ignore_index=True)
        val = pd.concat((positives[(3+run)%5],negatives[(3+run)%5]),ignore_index=True)
        test = pd.concat((positives[(4+run)%5],negatives[(4+run)%5]),ignore_index=True)
        X_train,Y_train,X_val,Y_val = encode(training,K, c_comp=c_comp, local_hp=local_hp, local_hp2 = local_hp2, K2=K2,win=win,ch=ch,features_filter=features_filter), training['class'], encode(val,K, c_comp=c_comp, local_hp=local_hp, local_hp2 = local_hp2, K2=K2,win=win,ch=ch,features_filter=features_filter), val['class']
        for gamma in gamma_list:
            for C in C_list:
                model = make_pipeline(svm.SVC(C=C, kernel='rbf', gamma=gamma))
                model.fit(X_train,Y_train)
                hyperparameter_table.append([K,gamma,C,matthews_corrcoef(Y_val,model.predict(X_val))])
                print([K,gamma,C,matthews_corrcoef(Y_val,model.predict(X_val))])
    best_hp = hyperparameter_table[np.array(hyperparameter_table).argmax(axis=0)[3]] 
    #test 
    best_K = int(float(best_hp[0]))
    best_C = float(best_hp[2])
    try: 
        best_gamma = float(best_hp[1])
    except ValueError: 
        best_gamma = best_hp[1]
    #
    X_train,Y_train,X_test,Y_test = encode(training,best_K, c_comp=c_comp, local_hp=local_hp, local_hp2 = local_hp2, K2=K2,win=win,ch=ch,features_filter=features_filter), training['class'], encode(test,best_K, c_comp=c_comp, local_hp=local_hp, local_hp2 = local_hp2, K2=K2,win=win,ch=ch,features_filter=features_filter), test['class']
    model = make_pipeline(svm.SVC(C=best_C, kernel='rbf', gamma=best_gamma))
    model.fit(X_train,Y_train)
    Y_test_pred= model.predict(X_test)
    mcc_test = matthews_corrcoef(Y_test,Y_test_pred)
    acc_test = accuracy_score(Y_test,Y_test_pred)
    prec_test = precision_score(Y_test,Y_test_pred)
    recall_test = recall_score(Y_test,Y_test_pred)
    #
    run_subsets = [run+1,((0+run)%5,(1+run)%5,(2+run)%5),(3+run)%5,(4+run)%5]
    run_subsets.extend(best_hp)
    run_subsets.extend([mcc_test,acc_test,prec_test,recall_test])
    cv_results.append(run_subsets)
    print('RUN '+str(run)+' ENDED---------------')
    print(run_subsets)
cv_results = pd.DataFrame(cv_results,columns=col_names)
print(model_combination)
print('MCC test avg: ', cv_results['MCC test'].mean())


[27, 8, 8, 0.880429787193724]
RUN 0 ENDED---------------
[1, (0, 1, 2), 3, 4, 27, 8, 8, 0.880429787193724, 0.8975403959215402, 0.981143193871538, 0.9080459770114943, 0.9080459770114943]
[27, 8, 8, 0.9000206790214517]
RUN 1 ENDED---------------
[2, (1, 2, 3), 4, 0, 27, 8, 8, 0.9000206790214517, 0.912308442774854, 0.9835197174808711, 0.9060773480662984, 0.9371428571428572]
[27, 8, 8, 0.9066210344849743]
RUN 2 ENDED---------------
[3, (2, 3, 4), 0, 1, 27, 8, 8, 0.9066210344849743, 0.894125359260882, 0.9805768098881695, 0.9127906976744186, 0.8971428571428571]
[27, 8, 8, 0.8975729586101995]
RUN 3 ENDED---------------
[4, (3, 4, 0), 1, 2, 27, 8, 8, 0.8975729586101995, 0.8667369437364472, 0.9758681577398469, 0.9036144578313253, 0.8571428571428571]
[27, 8, 8, 0.8605726332495074]
RUN 4 ENDED---------------
[5, (4, 0, 1), 2, 3, 27, 8, 8, 0.8605726332495074, 0.9016887915571702, 0.9811542991755006, 0.8823529411764706, 0.9428571428571428]
N-COMP+HP2+C-COMP+FF
MCC test avg:  0.8944799866501787


In [5]:
cv_results = cv_results.round(3)
cv_results.to_csv('results/'+model_combination+'.tsv',sep='\t',index=False)
cv_results

,run,training,validation,testing,K,gamma,C,MCC val,MCC test,ACC,precision,recall
0,1,"(0, 1, 2)",3,4,27,8,8,0.880,0.898,0.981,0.908,0.908
1,2,"(1, 2, 3)",4,0,27,8,8,0.900,0.912,0.984,0.906,0.937
2,3,"(2, 3, 4)",0,1,27,8,8,0.907,0.894,0.981,0.913,0.897
3,4,"(3, 4, 0)",1,2,27,8,8,0.898,0.867,0.976,0.904,0.857
4,5,"(4, 0, 1)",2,3,27,8,8,0.861,0.902,0.981,0.882,0.943


In [6]:
results = get_results(cv_results,model_combination)
results.to_csv('results/CV_results.tsv',mode='a',sep='\t',header=False)
results 

,model,K,gamma,C,MCC val,MCC test,ACC,precision,recall
0,N-COMP+HP2+C-COMP+FF,27,8,8,0.89 ± (0.01),0.89 ± (0.01),0.98 ± (0.00),0.90 ± (0.01),0.91 ± (0.02)


In [7]:
X_train

,HP2 max,HP2 max pos,L1,HP2 avg,R1,W1,K1,K2,I1,D1,...,G2,Q1,N1,Y2,V1,N2,D2,P1,Y1,H1
0,0.814530,0.250,0.370370,0.598674,0.074074,0.000000,0.000000,0.009804,0.000000,0.037037,...,0.137255,0.037037,0.000000,0.009804,0.000000,0.009804,0.049020,0.037037,0.000000,0.074074
1,0.805128,0.050,0.222222,0.548335,0.037037,0.000000,0.074074,0.019231,0.037037,0.000000,...,0.288462,0.037037,0.000000,0.038462,0.074074,0.048077,0.009615,0.000000,0.037037,0.037037
2,0.821368,0.075,0.333333,0.588948,0.037037,0.000000,0.037037,0.090909,0.000000,0.037037,...,0.077922,0.037037,0.000000,0.025974,0.037037,0.000000,0.038961,0.074074,0.000000,0.000000
3,0.810256,0.675,0.074074,0.522134,0.185185,0.074074,0.000000,0.028351,0.037037,0.037037,...,0.090206,0.000000,0.000000,0.028351,0.000000,0.036082,0.059278,0.111111,0.000000,0.000000
4,0.773504,0.250,0.185185,0.593015,0.037037,0.000000,0.000000,0.051630,0.000000,0.037037,...,0.046196,0.000000,0.000000,0.038043,0.148148,0.048913,0.062500,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5090,0.654701,0.525,0.000000,0.473268,0.111111,0.000000,0.000000,0.037946,0.074074,0.037037,...,0.058036,0.074074,0.037037,0.015625,0.000000,0.033482,0.062500,0.111111,0.000000,0.000000
5091,0.471795,0.450,0.000000,0.391748,0.074074,0.074074,0.074074,0.055482,0.000000,0.074074,...,0.058124,0.000000,0.037037,0.025099,0.037037,0.033025,0.067371,0.074074,0.000000,0.000000
5092,0.660684,0.075,0.111111,0.436988,0.111111,0.000000,0.000000,0.039106,0.037037,0.000000,...,0.055866,0.074074,0.037037,0.016760,0.074074,0.011173,0.039106,0.037037,0.000000,0.037037
5093,0.564957,0.400,0.111111,0.439464,0.074074,0.000000,0.222222,0.041789,0.037037,0.111111,...,0.057827,0.000000,0.000000,0.016264,0.037037,0.031398,0.055568,0.074074,0.000000,0.000000
